### **Ensemble Retriever**

`Ensemble Retriever`는 여러 검색기를 결합해서 결과를 제공하는 앙상블 기법을 이용한 Retriever

다양한 검색 알고리즘의 장점을 활용하여 단일 알고리즘보다 더 나은 성능 제공 

1. 여러 검색기 통합 : 다양한 유형의 검색기를 입력으로 받아 결과 결합
2. 결과 재순위화 : `Reciprocal Rank Fusion` 알고리즘을 사용하여 결과 순위 조정
3. 하이브리드 검색 : `Sparse retriever` 과 `Dense Retriever`를 결합하여 사용

`Sparse retriever` : 키워드 검색 기반 효과적

`Dense retriever` : 의미적 유사성 기반 검색 효과적 

In [4]:
# 각 검색기(BM25Retreiver, FAISS)를 결합 
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 샘플 문서 리스트
doc_list = [
    "I like apples",
    "I like apple company",
    "I like apple's iphone",
    "Apple is my favorite company",
    "I like apple's ipad",
    "I like apple's macbook",
]

In [ ]:
# bm25 검색기 생성
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 1  # 검색 개수 1

# 임베딩 정의(FAISS는 임베딩 생성 => 벡터 생성 => 검색기 생성 순으로 가야함)
embedding = OpenAIEmbeddings(model = 'text-embedding-3-small')

# FAISS 벡터 저장소 생성
faiss_store = FAISS.from_texts(doc_list, embedding)

# FAISS 검색기 생성
faiss_retriever = faiss_store.as_retriever(search_kwargs = {'k': 1})

# EnsembleRetriever 생성
ensemble_retriever = EnsembleRetriever(
    retrievers = [bm25_retriever, faiss_retriever],
    weights = [0.7, 0.3]  # 각 검색기의 가중치
)

`ensemble_retriever` 객체의 `get_relevant_documents()` 메서드를 호출하여 관련성 높은 문서를 검색

In [19]:
# 검색 결과 문서 반환 
query = 'my favorite fruit is apple'
ensemble_result = ensemble_retriever.invoke(query)
bm25_retriever_result = bm25_retriever.invoke(query)
faiss_retriever_result = faiss_retriever.invoke(query)

print('[EnsembleRetriever]')
for doc in ensemble_result:
    print(f'검색 결과 : {doc.page_content}')
    
print('[BM25Retriever]')
for doc in bm25_retriever_result:
    print(f'검색 결과 : {doc.page_content}')
    
print('[FAISSRetriever]')
for doc in faiss_retriever_result:
    print(f'검색 결과 : {doc.page_content}')

[EnsembleRetriever]
검색 결과 : Apple is my favorite company
검색 결과 : I like apple company
검색 결과 : I like apples
[BM25Retriever]
검색 결과 : Apple is my favorite company
검색 결과 : I like apple company
[FAISSRetriever]
검색 결과 : I like apples
검색 결과 : Apple is my favorite company
